In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper

In [2]:
pd.options.display.max_columns = 40

I received some excel files from my father-in-law containing information about electricity consumption in Ontario. He looked up the data [on the web](http://www.webroots.ca/static/ontarioelectricity/ontarioelectricity.html) three times daily and manually entered it into a spreadsheet.  There are three files in xlsx format. File 1 is observations from 2018, file 2 is observations from 2019, and file 3 is 1000 observations starting from August 2018 (approximately 13 months).

Loaded files in LibreOffice Calc to preview data. 

Most date entries are in dd-mmm format but a few are in dd-mmm-yy.

Column headers span two rows: row 1 is name and row 2 is units. Header in file 2 is an image instead of text.

File 3 contains two sheets with data. Looks like 1st sheet is a copy of the 2018 data and 2nd sheet is combined data from 2018-08-25 through 2019-09-30.

I see at least one data entry error (cell F12 in file 1).

Files 1 and 3 contain summary statisitics:
Three rows of stats at the bottom of the sheet: min, max, and avg of each column.
The two rightmost columns show average daily totals per month, and name of the month.

![](images/spreadsheet.png)

Fixed headers to contain both name and units in one row.

Set the format of all the date columns to `yyyy-mm-dd`.

Some of the numbers in the data contain commas (cell E6 and all of column R). Fixed by setting excel number format to `0.0` for percent columns and `0` for MW columns.

Deleted summary statistics.

Exported all four sheets as CSV.

Diffed the exported csv files. The extra sheet in file 3 is an unchanged copy of the data from file 1. Generated checksums to verify.

In [3]:
!diff data/electricity1.csv data/electricity3a.csv
!cksum data/*.csv

1070850776 26838 data/electricity1.csv
4245893866 90971 data/electricity2.csv
3911391547 96667 data/electricity3.csv
1070850776 26838 data/electricity3a.csv
2750527202 121891 data/electricity4.csv


In [4]:
df1 = pd.read_csv('data/electricity1.csv') # 2018 data
df2 = pd.read_csv('data/electricity2.csv') # 2019 data
df3 = pd.read_csv('data/electricity3.csv') # ~1000 observations starting August 2018

In [5]:
df4 = df1.append(df2)
df4.to_csv('data/electricity4.csv', index=False) # all data in one file

In [6]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)

(293, 18)
(1018, 18)
(1075, 18)
(1311, 18)


So now we've got all 1311 records in one data frame, `df4`. Let's take a look at these values and see what's up with the percentages. There were no formulas in the excel files so I'm thinking these values may have been calculated by hand, or maybe transcribed from the website. In any case they are probably redundant since we have the actual MW values. Let's check the math to make sure.

In [7]:
# np.where(condition, [x, y])
#df = df4.dropna()
#df['Total %'] = df['Nuclear %'] + df['Hydro %'] + df['Gas %'] + df['Wind %'] + df['Solar %'] + df['Biofuel %'] + df['Nuc.+Hyd. %']

In [8]:
df = pd.read_csv('data/electricity1.csv', parse_dates={'iso8601': [0, 1, 2]}, 
                  dtype={'Nuclear %': np.float64,
                         'Nuclear MW': np.int64,
                         'Hydro %': np.float64,
                         'Hydro MW': np.int64,
                         'Gas %': np.float64,
                         'Gas MW': np.int64,
                         'Wind %': np.float64,
                         'Wind MW': np.int64,
                         'Solar %': np.float64,
                         'Solar MW': np.float64,
                         'Biofuel %': np.object, # fix me
                         'Biofuel MW': np.float64,
                         'Nuc.+Hyd. %': np.float64,
                         'Nuc.+Hyd. MW': np.int64,
                         'Total MW': np.int64
                        })

In [9]:
df['Total %'] = df['Nuclear %'] + df['Hydro %'] + df['Gas %'] + df['Wind %'] + df['Solar %']

In [10]:
df.head()

,iso8601,Nuclear %,Nuclear MW,Hydro %,Hydro MW,Gas %,Gas MW,Wind %,Wind MW,Solar %,Solar MW,Biofuel %,Biofuel MW,Nuc.+Hyd. %,Nuc.+Hyd. MW,Total MW,Total %
0,2018-08-25 13:45:00,61.0,11017,17.2,3106,6.2,1120,14.5,2619,2.0,361.0,NaN,NaN,68.2,14123,18060,100.9
1,2018-08-25 20:45:00,60.9,11084,22.0,4004,5.6,1019,11.5,2093,0.0,NaN,NaN,NaN,82.9,15088,18200,100.0
2,2018-08-26 08:45:00,72.3,11058,18.7,2860,4.5,688,4.5,688,0.0,NaN,NaN,NaN,91.0,13918,15294,100.0
3,2018-08-26 12:45:00,66.6,11066,19.8,3262,10.3,1709,2.4,396,1.2,192.0,NaN,NaN,86.4,14328,16625,100.3
4,2018-08-26 16:45:00,60.1,11047,20.0,3674,16.0,2948,2.5,466,1.3,238.0,NaN,NaN,80.1,14721,18373,99.9


In [11]:
target = 'Total MW'
X = df4.drop(target, axis=1)
y = df4[target]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
mapper = DataFrameMapper([
    (['Date'], None), # [CategoricalImputer(), LabelBinarizer()]
    (['Day'], None),  # [CategoricalImputer(), LabelBinarizer()]
    (['Hour'], None), # [CategoricalImputer(), LabelBinarizer()]
    (['Nuclear %', 'Hydro %', 'Gas %', 'Wind %', 'Solar %', 'Biofuel %', 'Nuc.+Hyd. %', 'Nuc.+Hyd. MW', 'Total MW'],
     [SimpleImputer(), StandardScaler()]),
    (['Nuclear MW', 'Hydro MW', 'Gas MW', 'Wind MW', 'Solar MW', 'Biofuel MW'], [SimpleImputer(), StandardScaler()]),
    ([], [SimpleImputer(), StandardScaler()])
    ], df_out=True
)

In [13]:
#Z_train = mapper.fit_transform(X_train)
#Z_test = mapper.transform(X_test)